In [1]:
from selenium import webdriver
import time
import requests
import shutil
import os
import hashlib
import io
from PIL import Image  
import PIL
import shutil 
from webdriver_manager.chrome import ChromeDriverManager
import cv2
wd =  webdriver.Chrome(ChromeDriverManager().install())
# wd = webdriver.Chrome(executable_path=DRIVER_PATH)
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(10)
#             return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,url:str):
    size = 128, 128
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB').resize((512,512))
        print(image)
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

SAVE_FOLDER = 'images'
if not os.path.exists(SAVE_FOLDER):
    os.mkdir(SAVE_FOLDER)        
        
def search_and_download(search_term:str,wd,target_path='./images',number_images=500):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    
    res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
#     wd.close()
#     print(len(res))
    for elem in res:
        try:
            persist_image(target_folder,elem)
        except Exception as e:
            print(f"ERROR - Could not save  - {e}")
            
      
search_and_download("coffeemaker", wd=wd ,number_images=500)
# search_and_download("coffeemaker", wd=wd ,number_images=300) 
file_list = "images/"
save_path = "single_folder"
if not os.path.exists(save_path):
    os.mkdir(save_path)  
list_images = []
dir_list = os.listdir(file_list)
for directory in dir_list:
    images_path = os.listdir(os.path.join(file_list,directory))
    for image in images_path:
        list_images.append(os.path.join(os.path.join(file_list,directory) , image))
                

for i in range(len(list_images)):
    image_req = Image.open(list_images[i])
    image_req.save(f"single_folder/{i}.jpg")

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\shubh\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


 
Found: 100 search results. Extracting links from 0:100
Found: 162 image links, looking for more ...
Found: 312 search results. Extracting links from 100:312
Found: 502 image links, done!
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF77D508>
SUCCESS - saved https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F37%2F2019%2F02%2F27202943%2Fpink-coffee-mug-under-keurig-661fa4e2_crop.jpg - as ./images\coffeemaker\21218b1e01.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF78E808>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR6qCdely-Y2LJs5hQ0ECvOez5SHgz2UzkIvg&usqp=CAU - as ./images\coffeemaker\313a222681.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF795688>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRmUCdscPfOqextAeO8YZ4wKtmdir56d4nqxw&usqp=CAU - as ./images\coffeemaker\fcd7cd16c8.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF78EB

<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF78E808>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT3Lb7Jy4WTJyiIM1IKm_wcow-7Eni4MZrGgg&usqp=CAU - as ./images\coffeemaker\f24fbc8f8d.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF6BBD48>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSwLT5A7BEcD0v1dSuMuoWCj16yn9MGhVqskQ&usqp=CAU - as ./images\coffeemaker\454aff429b.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF77DC48>
SUCCESS - saved https://images.heb.com/is/image/HEBGrocery/001773753 - as ./images\coffeemaker\b5c4dd3428.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF6CDBC8>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRYyfqSPFMhQyHXGsF-qMLsDmhuv3MpD4ekPw&usqp=CAU - as ./images\coffeemaker\d748eec3ff.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF787608>
SUCCESS - saved https://crdms.images.consumerreports.org/c_lfill,w_598/prod/products/cr/models/400

c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF78DFC8>
SUCCESS - saved https://assets.croma.com/medias/sys_master/images/images/h4c/h99/8912874405918/-1200Wx1200H-186714-1-300Wx300H.png - as ./images\coffeemaker\b1eb107552.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FE273C08>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTLtbaijzRDx9a-lDfFk2CTuW4zPQQGLjPGvw&usqp=CAU - as ./images\coffeemaker\b50264a2ad.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF6BB548>
SUCCESS - saved https://target.scene7.com/is/image/Target/GUEST_2e47eb6e-1011-414c-b621-8c936b9e185a?wid=488&hei=488&fmt=pjpeg - as ./images\coffeemaker\0309dad356.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF77DC48>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRpG6wvhm_xYbgFAwXTLj7oE40FI4csbAjmAQ&usqp=CAU - as ./images\coffeemaker\92c97f4a50.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168F7210F48>
SUCCESS - saved https://con

...

<PIL.Image.Image image mode=RGB size=512x512 at 0x168880BAF08>
SUCCESS - saved https://cdn0.vox-cdn.com/hermano/verge/product/image/8061/Standalone_1175X1290.jpg - as ./images\coffeemaker\62757455f2.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880AFFC8>
SUCCESS - saved https://www.reliancedigital.in/medias/Wonderchef-63152587-Coffe-Makers-Grinders-491473359-i-1-1200Wx1200H-300Wx300H?context=bWFzdGVyfGltYWdlc3w3MTc4fGltYWdlL2pwZWd8aW1hZ2VzL2hhOS9oZWMvOTA4MzAzMTc4MTQwNi5qcGd8NWFhNTM4NzkyZTNhOTE5YWRlMGY1YjIxNDlkMWJjMDBhYzRlMThkZGI0ZDgyMTg4N2U2Mjk4ODBiYzQ4MzcyZg - as ./images\coffeemaker\d9f95f952d.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x16888019988>
SUCCESS - saved https://img1.exportersindia.com/product_images/bc-full/2018/8/4827941/coffee-maker-1535532560-4242430.jpeg - as ./images\coffeemaker\b58f615969.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF6ED288>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS1dBvLR6nt9oiGjdbsl8

<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF796BC8>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRQmqRn75xCGHENgY20y07xkfrbBr4x_AwFQw&usqp=CAU - as ./images\coffeemaker\10bda7432d.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880AFC48>
SUCCESS - saved https://mobileimages.lowes.com/product/converted/100248/1002487722.jpg?size=pdhi - as ./images\coffeemaker\3bdf186211.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x16888019888>
SUCCESS - saved https://assets.croma.com/medias/sys_master/images/images/h8d/hdb/8810547544094/185818_pjpeg.jpg - as ./images\coffeemaker\c1d5664f05.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168F5D8B388>
SUCCESS - saved https://rukminim1.flixcart.com/image/612/612/jdt4n0w0/coffee-maker/x/m/y/black-decker-bxcm1201in-original-imaf2mpffj7ujy3h.jpeg?q=70 - as ./images\coffeemaker\3bc6fa31a8.jpg
ERROR - Could not save https://www.williams-sonoma.com/wsimgs/rk/images/dp/wcm/201947/0014/cuisinart-perfect

<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF795AC8>
SUCCESS - saved https://cdn.thewirecutter.com/wp-content/uploads/2019/05/cheapcoffeemakers-lowres-5784-570x380.jpg - as ./images\coffeemaker\8d666517ff.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF796908>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcREfiTSkmqeItHKO1Sl6QgFqCc--kExBKg2sQ&usqp=CAU - as ./images\coffeemaker\97c26951be.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FE273E88>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSs_aTqMuoSaKgSYZ1TfYCwDXDNoHYJhFY2Gw&usqp=CAU - as ./images\coffeemaker\ef1aba6ff7.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x1688804FEC8>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRAzdb_dhTHjZj3EZ7CxXWYrcQbW4pfZrOkJg&usqp=CAU - as ./images\coffeemaker\72d41ecd77.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880A9D08>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/i

<PIL.Image.Image image mode=RGB size=512x512 at 0x168880B2AC8>
SUCCESS - saved https://www.telegraph.co.uk/content/dam/food-and-drink/2019/09/19/sage-oracle-touch-fully-automatic-bean-to-cup-coffee-machine_trans_NvBQzQNjv4BqGyDzj1Q6wzTf2DvQ0Fu1dPtGkmS9BaS-Y4tHMtzgyAM.JPG - as ./images\coffeemaker\7e62c04bc9.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF795688>
SUCCESS - saved https://pisces.bbystatic.com/image2/BestBuy_US/images/products/6287/6287390_sd.jpg - as ./images\coffeemaker\625597655a.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880B7F88>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSToe_Aft-g460OZ4RKvHtdTOVRLcPqNiVDJw&usqp=CAU - as ./images\coffeemaker\853faae5d0.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168FF795688>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR0zTAAOh7kIDbJIYNZMRukZcYa8jvzhnudWA&usqp=CAU - as ./images\coffeemaker\5e2b3b31f4.jpg
<PIL.Image.Image image mode=RGB size=512

<PIL.Image.Image image mode=RGB size=512x512 at 0x168880BA808>
SUCCESS - saved https://3.imimg.com/data3/PR/NM/GLADMIN-164121/filter-coffee-maker-500x500.jpg - as ./images\coffeemaker\420832ce5d.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880B2FC8>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS3SdTSrB1YTzq0HS-hMMiVfnYSk032VTUX3Q&usqp=CAU - as ./images\coffeemaker\0aa3ecc545.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880A9D08>
SUCCESS - saved https://specials-images.forbesimg.com/imageserve/5e9e1512736d170006fed62f/960x0.jpg?fit=scale - as ./images\coffeemaker\9cd0fb4802.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880AEE48>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRrqknFEH2mcRT95nx9YVSYNbVemqQsZ6updA&usqp=CAU - as ./images\coffeemaker\44428a18f0.jpg
<PIL.Image.Image image mode=RGB size=512x512 at 0x168880B2AC8>
SUCCESS - saved https://m.economictimes.com/thumb/msid-62308131,width-1200,heigh

In [65]:
file_list = "images/"

save_path = "single_folder"
if not os.path.exists(save_path):
    os.mkdir(save_path)  
list_images = []
dir_list = os.listdir(file_list)
for directory in dir_list:
    images_path = os.listdir(os.path.join(file_list,directory))
    for image in images_path:
        list_images.append(os.path.join(os.path.join(file_list,directory) , image))
                

for i in range(len(list_images)):
    image_req = Image.open(list_images[i])
    image_req.save(f"single_folder/{i}.jpg")